In [22]:
import pandas as pd
import glob

In [23]:
def concatenate_gpt(files):
    dfs = []
    for file in files:
        raw_df = pd.read_csv(file)
        df = pd.DataFrame({'text': [str(t).strip() for t in raw_df['text']],
                       'generated': 1})
        dfs.append(df)

    gpt = pd.concat(dfs, ignore_index=True)
    gpt.to_csv('gpt.csv', index=False)
    return gpt


In [24]:
concatenate_gpt(glob.glob('gpt_data/*.csv'))

,text,generated
0,"Dawn on its way to victory, but where does it ...",1
1,Drew Angerer/Getty Images\n\nThere are many th...,1
2,"In my last post, I gave a post on the use of t...",1
3,Brought to you by:\n\nKFC\n\nHogan's Day\n\nSo...,1
4,"It's time for the first time since 2008, and w...",1
...,...,...
9995,"alabamer, philly pix, pestle, live nectar, dan...",1
9996,Ballistics System Summary\n\nA linear three-di...,1
9997,http://dailytarbadilla.com/policies/opinions/2...,1
9998,"Harmony in that regard is momentous'"", Ocaden ...",1


In [25]:
def process_nyt():
    dfs = []

    files = glob.glob('nyt_data/*.csv')

    for file in files:
        raw_df = pd.read_csv(file)
        df = pd.DataFrame({'text': [str(t).strip() for t in raw_df['abstract']],
                       'generated': 0})
        dfs.append(df)

    nyt_scraped_files = glob.glob('../nyt_scraper/data/*.txt')
    for file in nyt_scraped_files:
        lines = open(file).readlines()
        df = pd.DataFrame({'text': [l.strip() for l in lines],
                       'generated': 0})
        dfs.append(df)
    human = pd.concat(dfs, ignore_index=True)
    human.to_csv('human.csv', index=False)
    return human

In [26]:
process_nyt()

,text,generated
0,Congress could do much more to protect America...,0
1,Christina Iverson and Jeff Chen ring in the Ne...,0
2,"All year long, Earth passes through streams of...",0
3,"Never miss an eclipse, a meteor shower, a rock...",0
4,A year full of highs and lows in space just en...,0
...,...,...
72413,Mr. Sri-Kumar finds that a tall order and enco...,0
72414,He would own Treasury bonds and bond surrogate...,0
72415,Other choices include stocks in emerging econo...,0
72416,"Mr. Atteberry encourages caution, too. Whateve...",0
